In [2]:
%cd ..
%ls

/Users/matusbojko/Desktop/dp-project/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/Users/matusbojko/Desktop/dp-project
LICENSE           main.ipynb        project/
README.md         notebooks/        requirements.txt
data/             prepare.sh*       run/


In [3]:
from project.datamodule import BaseDataSets, RandomGenerator,TwoStreamBatchSampler,patients_to_slices
from project.utils import worker_init_fn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib import colors
from torchvision import transforms
from scipy.ndimage import zoom
from medpy import metric
import torch
import random
import numpy as np

In [3]:

def show_sample(image,label):

    # Convert PyTorch tensor (image) to NumPy array and squeeze the channel dimension if necessary
    if isinstance(image, np.ndarray):
        image_np = image  # Already a NumPy array
    else:
        image_np = image.numpy()  # Convert PyTorch tensor to NumPy array
    
    # Squeeze the singleton channel dimension (1, 254, 254) -> (254, 254)
    if image_np.shape[0] == 1:
        image_np = np.squeeze(image_np, axis=0)  # Remove the channel dimension

    # Define a color map for the labels: 0 (background), 1, 2, 3
    cmap = colors.ListedColormap(['black', 'red', 'green', 'blue'])  # Customize these colors as needed
    bounds = [0, 1, 2, 3, 4]  # Boundaries for the label values
    norm = colors.BoundaryNorm(bounds, cmap.N)

    plt.figure(figsize=(10, 5))

    # Plot the image (assuming it's grayscale)
    plt.subplot(1, 2, 1)
    plt.imshow(image_np, cmap='gray')  # Display image as grayscale
    plt.title('Image')

    # Plot the label with the custom color map
    plt.subplot(1, 2, 2)
    plt.imshow(label, cmap=cmap, norm=norm)  # Color the label based on defined cmap
    plt.colorbar(ticks=[0, 1, 2, 3])  # Show the color bar with label values
    plt.title('Colored Segmentation Mask')

    plt.show()

In [4]:
random.seed(1337)
np.random.seed(1337)
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)
torch.mps.manual_seed(1337)


db_train = BaseDataSets(base_dir="./data/ACDC", split="train", num=None, transform=transforms.Compose([
    RandomGenerator([256, 256])
]))

db_val = BaseDataSets(base_dir="./data/ACDC", split="val")

total_slices = len(db_train)
labeled_slice = patients_to_slices("ACDC", 7)
labeled_idxs = list(range(0, labeled_slice))
unlabeled_idxs = list(range(labeled_slice, total_slices))

batch_sampler = TwoStreamBatchSampler(labeled_idxs, unlabeled_idxs, 24, 24-12)

trainloader = DataLoader(db_train,num_workers=4,batch_size=1, pin_memory=True, worker_init_fn=worker_init_fn)
valloader = DataLoader(db_val, batch_size=1, shuffle=False,num_workers=1)




| Labeled samples: 5.0%


In [16]:
count = 0
max = 10000
min = 0
for i_batch, sampled_batch in enumerate(trainloader):
    volume_batch, label_batch = sampled_batch['image'], sampled_batch['label']
    if i_batch == 0:
        max = volume_batch[0]
        min = volume_batch[0]
    
    if volume_batch[0] > max:
        max = volume_batch[0]
    
    if volume_batch[0] < min:
        min = volume_batch[0]
    
print(max)
print(min)







tensor(512)
tensor(154)


In [17]:
def test_single_volume_ds(image, label):
    image, label = image.squeeze(0).cpu().detach().numpy(), label.squeeze(0).cpu().detach().numpy()
    #(1, 10, 256, 224) squeeze-> (10,256, 224) -> detach (gradient not longer computed for tensor)

    prediction = np.zeros_like(label)

    for ind in range(image.shape[0]): 
        slice = image[ind, :, :] # (256, 224)
        x, y = slice.shape[0], slice.shape[1] # 256, 224
        slice = zoom(slice, (256 / x, 256 / y), order=0) # 1, 1.14 -> after zoom (256,256)
        print(slice.shape)
        input = torch.from_numpy(slice).unsqueeze(0).unsqueeze(0).float().to("mps") # ADD CUDA or TO DEVICE -> ([1, 1, 256, 256])
        print(input.shape)
        
        net.eval()
        with torch.no_grad():
            output_main, _, _, _ = net(input)
            out = torch.argmax(torch.softmax(
                output_main, dim=1), dim=1).squeeze(0)
            out = out.cpu().detach().numpy()
            pred = zoom(out, (x / patch_size[0], y / patch_size[1]), order=0)
            prediction[ind] = pred
        metric_list = []
        for i in range(1, classes):
            metric_list.append(calculate_metric_percase(
                prediction == i, label == i))
        return metric_list

def calculate_metric_percase(pred, gt):
    pred[pred > 0] = 1
    gt[gt > 0] = 1
    if pred.sum() > 0:
        dice = metric.binary.dc(pred, gt)
        hd95 = metric.binary.hd95(pred, gt)
        return dice, hd95
    else:
        return 0, 0

In [10]:
for i_batch, sampled_batch in enumerate(trainloader):
    volume_batch, label_batch = sampled_batch['image'], sampled_batch['label']

    # print(volume_batch.shape)
    # print(label_batch.shape)
    # print(i_batch)

    show_sample(volume_batch[0],label_batch[0])


    if i_batch == 15:
        break

#     torch.Size([24, 1, 256, 256])
# torch.Size([24, 256, 256])

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/matusbojko/Desktop/dp-project/.venv/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/Users/matusbojko/Desktop/dp-project/.venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/matusbojko/Desktop/dp-project/.venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/matusbojko/Desktop/dp-project/project/datamodule.py", line 46, in __getitem__
    sample = self.transform(sample)
TypeError: 'NoneType' object is not callable


In [1]:
import torch
print(torch.backends.cudnn.version())

None
